In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import tqdm
import json

In [2]:
apiKey = "RGAPI-8fb76f45-f6fe-494d-823c-7c22227b4dae"

In [3]:
with open("games_id.json","r") as f:
    games_id = json.load(f)
print(len(games_id))
print(games_id[0])

3607
EUN1_2339766487


In [4]:
def get_game_details_from_game_id(game_id, api):
    url = "https://europe.api.riotgames.com/tft/match/v1/matches/"+game_id+"?api_key="+api
    response = requests.get(url).json()['info']['participants']
    # print(np.array(response).shape)
    ans = []
    for i in response:
        player = []
        for champs in i['units']:
            # print(champs)
            # print(champs['name'], champs['items'], champs['tier'])
            player.append([champs['name'], champs['items'], champs['tier']])
        # print(player)
        ans.append(player)
    return ans

In [5]:

games_id = list(games_id)
all_games = []
starttime = time.time()
for i in tqdm.tqdm(range(len(games_id)//20)):    
    starttime = time.time()
    for j in range(20):
        try:
            all_games.append(get_game_details_from_game_id(games_id[i*20+j], apiKey))
        except:
            pass
    time.sleep(25.0 - (time.time()-starttime))

print()
print(len(all_games))

with open('games.json', 'w') as f:
    json.dump(all_games, f)

100%|██████████████████████████████████████████████████████████████████████████████| 180/180 [1:15:00<00:00, 25.00s/it]



3599
